# 2.7. Inspecting metrics with Tensorboard

## Data loading and preprocessing

In [0]:
import pandas as pd
from google.colab import files
#uploaded = files.upload()
data = pd.read_csv('/content/preprocessed_data.csv')
data = data.sample(frac=1) # a bit weird
data.head()

,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,...,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
1862,Red,False,3,2.0,0.0,0.0,17.500000,13.500000,11.750000,8.750000,77.750000,27.750000,20.000000,11.000000,89.250000,31.500000,0.500000,2.750000,2.500000,0.750000,0.500000,109.500000,47.500000,0.452500,0.000000,0.250000,0.000000,0.000000,144.250000,80.500000,2.0,2.0,7.250000,5.250000,1.750000,1.000000,53.500000,19.000000,16.500000,7.000000,...,75.750000,33.000000,10.0,723.250000,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,190.50,203.20,170.0,30.0,26.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
3339,Red,False,3,0.0,1.0,0.0,6.333333,2.333333,3.333333,1.333333,42.666667,9.000000,10.333333,4.000000,47.333333,10.333333,0.333333,2.666667,1.666667,2.000000,0.333333,56.333333,14.333333,0.340000,1.333333,1.666667,1.333333,0.666667,96.666667,52.666667,1.0,1.0,2.000000,1.000000,6.333333,3.333333,37.666667,9.666667,17.666667,9.000000,...,168.000000,45.000000,3.0,900.000000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,182.88,190.50,205.0,25.0,26.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1759,Blue,False,3,0.0,3.0,0.0,2.909091,1.636364,3.181818,1.909091,40.636364,11.454545,12.363636,8.363636,50.181818,17.181818,0.272727,3.090909,2.909091,4.363636,0.000000,56.181818,21.727273,0.408182,1.181818,6.636364,2.090909,0.334545,78.272727,42.000000,5.0,3.0,6.090909,4.181818,6.454545,4.454545,46.909091,20.272727,6.363636,4.545455,...,68.875000,37.625000,22.0,775.125000,0.0,0.0,1.0,3.0,0.0,2.0,0.0,6.0,190.50,198.12,205.0,32.0,32.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2962,Red,False,3,1.0,0.0,0.0,3.333333,3.166667,7.166667,5.500000,38.166667,9.666667,1.666667,1.166667,39.666667,10.666667,0.666667,4.000000,2.500000,0.000000,0.000000,47.000000,16.333333,0.460000,0.000000,0.166667,0.000000,0.000000,75.666667,44.333333,1.0,4.0,1.166667,0.833333,2.666667,1.666667,33.833333,11.833333,6.833333,3.333333,...,49.142857,24.071429,32.0,587.857143,5.0,0.0,0.0,2.0,7.0,1.0,1.0,11.0,185.42,190.50,185.0,32.0,34.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1082,Blue,False,3,0.0,1.0,0.0,16.714286,12.571429,10.142857,7.571429,89.428571,26.285714,14.142857,10.000000,96.142857,30.428571,0.000000,0.857143,0.857143,1.714286,0.142857,113.714286,43.857143,0.358571,1.000000,5.714286,2.142857,0.348571,137.714286,64.714286,4.0,2.0,10.000000,7.000000,6.000000,4.142857,72.000000,31.428571,3.571429,2.714286,...,53.400000,28.400000,11.0,597.200000,0.0,0.0,0.0,0.0,0.0,3.0,1.0,4.0,180.34,185.42,170.0,23.0,21.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,

In [0]:
data['Winner'] = data['Winner'].map(lambda x: 1 if x == 'Red' else 0)
data['title_bout'] = data['title_bout'].map(lambda x: 1 if x == 'True' else 0)

In [0]:
train_size = int(0.8*len(data))
features = data.drop(columns=['Winner'])
targets = data['Winner']
X_train, X_test = features.values[:train_size, :], features.values[train_size:, :]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
#sns.pairplot(data)

In [0]:
corr = data.corr()
cmap = sns.diverging_palette(250, 10, as_cmap=True)
plt.figure(figsize=(40, 80))
sns.heatmap(corr[['Winner']], cmap=cmap, vmax=.3,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},
            annot=True)

## Tensorflow ANNs

In [0]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import os, datetime
%reload_ext tensorboard

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu), # x if x > 0 else alpha*x 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3)
])

red = len(y_train[y_train>0])
blue = len(y_train) - red
total = len(y_train)
weight_for_red = total / (2 * red) 
weight_for_blue = total / (2 * blue)
class_weight = {0: weight_for_blue, 1: weight_for_red}
print(class_weight)

adam_optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=adam_optimizer,
    loss='binary_crossentropy',
    metrics=[
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
    ]
)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(
    '/content/model-{epoch:02d}-{val_accuracy:.2f}.hdf5', 
    monitor='val_accuracy', 
    verbose=1,
    save_best_only=True, 
    save_weights_only=False,
    save_frequency=1)

logdir = os.path.join('tflogs2', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, 
                                                      histogram_freq=1,
                                                      profile_batch=0)

%tensorboard --logdir tflogs2

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model.fit(X_train_scaled, 
          y_train,
          class_weight=class_weight,
          batch_size=64,
          validation_split=0.1,
          callbacks=[tensorboard_callback],
          epochs=100,
          verbose=0)

In [0]:
import numpy as np
#model = tf.keras.models.load_model('/content/model-43-0.64.hdf5')
X_test_scaled = scaler.transform(X_test)
model.evaluate(X_test_scaled, y_test)
#np.round(model.predict(X_test))

In [0]:
!pip install --upgrade tensorflow